In [ ]:
import numpy as np
import pandas as pd
import itertools
from mass_function import MassFunction


In [ ]:
ALPHA = 0.1


In [ ]:
ROOT = ".."
df_data = pd.read_csv(f"{ROOT}/data/HEA_data.demo.csv", index_col=0)
ELEMENTS = np.array([
    "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu",
    "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag"])


In [ ]:
from ERS_misc import extract_evidence
alloy_i = ["Ag", "Fe"]
query_string = "Ag==1 and Fe==1 and n_element==2"
label_alloy_i = df_data.query(query_string).iloc[0]["Label"]
alloy_j = ["Cu", "Fe"]
query_string = "Cu==1 and Fe==1 and n_element==2"
label_alloy_j = df_data.query(query_string).iloc[0]["Label"]
evidence_1 = extract_evidence(frozenset(alloy_i), frozenset(alloy_j))
if evidence_1 is not None:
    combination_t, combination_v = evidence_1
    if label_alloy_i == label_alloy_j:
        mass_function_1 = MassFunction(source=[({"similar"}, 
                                               ALPHA)],
                                       coreset={"similar", "dissimilar"})
    else:
        mass_function_1 = MassFunction(source=[({"dissimilar"}, 
                                                ALPHA)],
                                       coreset={"similar", "dissimilar"})
    print(mass_function_1)


In [ ]:
alloy_i = ["Ag", "Pd", "V"]
query_string = "Ag==1 and Pd==1 and V==1 and n_element==3"
label_alloy_i = df_data.query(query_string).iloc[0]["Label"]
alloy_j = ["Cu", "Pd", "V"]
query_string = "Cu==1 and Pd==1 and V==1 and n_element==3"
label_alloy_j = df_data.query(query_string).iloc[0]["Label"]
evidence_2 = extract_evidence(frozenset(alloy_i), frozenset(alloy_j))
if evidence_2 is not None:
    combination_v, similar = evidence_2
    if label_alloy_i == label_alloy_j:
        mass_function_2 = MassFunction(source=[({"similar"}, 
                                                ALPHA)],
                                       coreset={"similar", "dissimilar"})
    else:
        mass_function_2 = MassFunction(source=[({"dissimilar"}, 
                                               ALPHA)],
                                       coreset={"similar", "dissimilar"})
    print(mass_function_2)


In [ ]:
combined_mass_function = mass_function_1.combine(mass_function_2)
print(combined_mass_function)


In [ ]:
import progressbar
nall = int(df_data.shape[0]*(df_data.shape[0]-1)/2)
bar = progressbar.ProgressBar(max_value=nall)
iteration = itertools.combinations(df_data.index.values, 2)
mass_functions = []
for i, (index_alloy_i, index_alloy_j) in enumerate(iteration):
    bar.update(i+1)
    df_alloy_i = df_data.loc[index_alloy_i]
    df_alloy_j = df_data.loc[index_alloy_j]
    alloy_i = ELEMENTS[df_alloy_i[ELEMENTS].values == 1].tolist()
    label_alloy_i = df_alloy_i["Label"]
    alloy_j = ELEMENTS[df_alloy_j[ELEMENTS].values == 1].tolist()
    label_alloy_j = df_alloy_j["Label"]
    evidence = extract_evidence(frozenset(alloy_i), frozenset(alloy_j))
    if evidence is not None:
        combination_t, combination_v = evidence
        if label_alloy_i == label_alloy_j:
            mass_function = MassFunction(source=[({"similar"},
                                                 ALPHA)],
                                         coreset={"similar", "dissimilar"})
        else:
            mass_function = MassFunction(source=[({"dissimilar"},
                                                  ALPHA)],
                                         coreset={"similar", "dissimilar"})
        mass_functions.append([combination_t, combination_v, mass_function])


In [ ]:
from ERS_misc import make_similarity_df
df_similarity, df_dissimilarity, df_unknown = make_similarity_df(ELEMENTS,
                                                           mass_functions)


In [ ]:
#結果の保存
import os
outputdir = "data_executed"
os.makedirs(outputdir, exist_ok=True)
df_similarity.to_csv(os.path.join(outputdir,"elements_similarity.csv"))
df_dissimilarity.to_csv(os.path.join(outputdir,"elements_dissimilarity.csv"))
df_unknown.to_csv(os.path.join(outputdir,"elements_unknown.csv"))

In [ ]:
from ERS_misc import plot_similarity_matrix
plot_similarity_matrix(df_similarity, ELEMENTS)


In [ ]:
SCALE = .1


In [ ]:
candidate = ["Fe", "Mn", "Co", "Ni"]


In [ ]:
from ERS_misc import extract_HEA_evidence
alloy_k = ["Co", "Cr", "Fe", "Ni"]
qstr = "combination=='{}'".format("".join(alloy_k))
label_alloy_k = df_data.query(qstr).iloc[0]["Label"]
evidence_1 = extract_HEA_evidence(frozenset(candidate), frozenset(alloy_k))
if evidence_1 is not None:
    combination_t, combination_v = evidence_1
    index_t = "|".join(sorted(combination_t))
    index_v = "|".join(sorted(combination_v))
    similar_score = df_similarity.loc[index_t, index_v]
    mass_function_1 = MassFunction(source=[({label_alloy_k},
                                    similar_score*SCALE)],
                                   coreset={"HEA", "not_HEA"})
    print(mass_function_1)


In [ ]:
alloy_k = ["Co", "Cr", "Mn"]
qstr = "combination=='{}'".format("".join(alloy_k))
label_alloy_k = df_data.query(qstr).iloc[0]["Label"]
evidence_2 = extract_HEA_evidence(frozenset(candidate), frozenset(alloy_k))
if evidence_2 is not None:
    combination_t, combination_v = evidence_2
    index_t = "|".join(sorted(combination_t))
    index_v = "|".join(sorted(combination_v))
    similar_score = df_similarity.loc[index_t, index_v]
    mass_function_2 = MassFunction(source=[({label_alloy_k},
                                    similar_score*SCALE)],
                                   coreset={"HEA", "not_HEA"})
    print(mass_function_2)


In [ ]:
combined_mass_function = mass_function_1.combine(mass_function_2)
print(combined_mass_function)
